Import Libraries

In [38]:
import pandas as pd
import numpy as np
import os
import yfinance as yf


Currency Codes

In [39]:
country_currency_codes_table = {
    'Burundi': 'BIF',
    'Benin': 'XOF',
    'Burkina Faso': 'XOF',
    'Central African Republic': 'XAF',
    "Côte d'Ivoire": 'XOF',
    'Cameroon': 'XAF',
    'Democratic Republic of the Congo': 'CDF',
    'Republic of Congo': 'XAF',
    'Comoros': 'KMF',
    'Eritrea': 'ERN',
    'Ethiopia': 'ETB',
    'Ghana': 'GHS',
    'Guinea': 'GNF',
    'The Gambia': 'GMD',
    'Guinea-Bissau': 'XOF',
    'Liberia': 'LRD',
    'Madagascar': 'MGA',
    'Mali': 'XOF',
    'Mozambique': 'MZN',
    'Mauritania': 'MRU',
    'Malawi': 'MWK',
    'Niger': 'XOF',
    'Rwanda': 'RWF',
    'Sudan': 'SDG',
    'Senegal': 'XOF',
    'Sierra Leone': 'SLL',
    'Somalia': 'SOS',
    'Sao Tomee and Principe': 'STN',
    'Chad': 'XAF',
    'Togo': 'XOF',
    'Tanzania': 'TZS',
    'Uganda': 'UGX',
    'Zambia': 'ZMW'
}


Read Data

In [40]:
#read in stata dataset as dataframe "country_code_df"
country_code_df = pd.read_stata("codesffactors_Mar23_2019.dta") 
country_code_df.replace('--', np.nan, inplace=True) #replace all entries that are '--' with NaN data value

country_code_df.head()

,countryname,wbctry,wbcode,wbregion,wbigroup2017,wblcat2017,hipc,emu,cemac,waemu,imfcode,imfweocode,imfweoiso,imfweoctry,region
0,Aruba,Aruba,ABW,Latin America & Caribbean,High income,NaN,0,0,0,0,NaN,314.0,ABW,Aruba,LAC
1,Afghanistan,Afghanistan,AFG,South Asia,Low income,IDA,1,0,0,0,512.0,512.0,AFG,Afghanistan,SAS
2,Angola,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,0,0,0,0,614.0,614.0,AGO,Angola,SSA
3,Anguilla,NaN,AIA,Latin America & Caribbean,NaN,NaN,0,0,0,0,312.0,NaN,NaN,NaN,LAC
4,Albania,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,0,0,0,0,914.0,914.0,ALB,Albania,ECA


In [41]:
region_classification_ls = country_code_df['wbregion'].unique() #get a list of region classifications
income_classification_ls = country_code_df['wbigroup2017'].unique() #get a list of income classifications
income_classification_ls = [x for x in income_classification_ls if not pd.isna(x)] # remove '--' from income_classification_ls

print(f"The region classifications are: {', '.join(country for country in region_classification_ls)}")
print(f"The income classifications are: {', '.join(country for country in income_classification_ls)}")

The region classifications are: Latin America & Caribbean, South Asia, Sub-Saharan Africa, Europe & Central Asia, Middle East & North Africa, East Asia & Pacific, North America
The income classifications are: High income, Low income, Lower middle income, Upper middle income


In [42]:
ssa_hipc_countries = country_code_df[(country_code_df['hipc'] == 1) & (country_code_df['wbregion'] == 'Sub-Saharan Africa')]
ssa_hipc_countries_ls = ssa_hipc_countries['imfweoctry'].to_list()

ssa_hipc_countries_ls

['Burundi',
 'Benin',
 'Burkina Faso',
 'Central African Republic',
 "Côte d'Ivoire",
 'Cameroon',
 'Democratic Republic of the Congo',
 'Republic of Congo',
 'Comoros',
 'Eritrea',
 'Ethiopia',
 'Ghana',
 'Guinea',
 'The Gambia',
 'Guinea-Bissau',
 'Liberia',
 'Madagascar',
 'Mali',
 'Mozambique',
 'Mauritania',
 'Malawi',
 'Niger',
 'Rwanda',
 'Sudan',
 'Senegal',
 'Sierra Leone',
 'Somalia',
 'São Tomé and Príncipe',
 'Chad',
 'Togo',
 'Tanzania',
 'Uganda',
 'Zambia']

Read in IMF's Government Finance Statistics

In [43]:
master_table = {}
folder_path = "expenditure-investment-data"

# Loop that goes through all files 'file_name' in the folder 'folder_path'
for file_name in os.listdir(folder_path):
  if file_name.endswith('.xlsx') and not file_name.startswith('~$'): #check to see if is spreadsheet .xlsx format
    raw_df = pd.read_excel(folder_path+ "/" + file_name) #read in .xlsx as dataframe
    master_table[file_name[:-5]] = raw_df #store dataframe as an entry in the master_table hash table

# master_table["Central African Republic"]
for key in master_table:
  print(master_table[key])

                                           Unnamed: 0   Unnamed: 1  \
0   Statement of Operations (Revenue, Expenditures...          NaN   
1                                      Country: Benin          NaN   
2                Sector: Budgetary central government          NaN   
3                             Unit: Domestic currency          NaN   
4                                     Scale: Billions          NaN   
5                                                 NaN          NaN   
6                                                 NaN  2010.000000   
7                    TRANSACTIONS AFFECTING NET WORTH          NaN   
8                                             Revenue   648.375468   
9                                               Taxes   536.250675   
10                               Social contributions    16.366616   
11                                             Grants    52.312694   
12                                      Other revenue    43.445484   
13                  

Filtering

In [44]:
#REMOVE "JUNK" ROWS AND CONVERT TO CORRECT SCALE (MILLION/BILLIONS)
for key in master_table: #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
  scale = master_table[key].iloc[4,0] #get if values have scale of billions or millions
  
  # Drop rows 0 to 5, reformatting indexes and column headers
  master_table[key] = master_table[key].drop([0, 1, 2, 3, 4, 5, 11]) 
  master_table[key] = master_table[key].reset_index(drop=True) 
  master_table[key].columns = master_table[key].iloc[0]
  master_table[key] = master_table[key][1:]
  master_table[key] = master_table[key].set_index(master_table[key].columns[0])
  master_table[key].columns = [int(float(col)) for col in master_table[key].columns]
  # master_table[key] = master_table[key].rename({4: 'Tax Grants'}, axis='index')
  # index_to_rename = master_table[key].index[master_table[key]['name'] == 'grants'][0]
  # master_table[key].at[index_to_rename, 'name'] = 'tax grants'


  # master_table[key] = master_table[key].fillna(0)

  if scale == "Scale: Billions":
    master_table[key] = master_table[key]* (10 ** 9) #multiple every value in dataframe by 1e9 if scale is billion
  elif scale == "Scale: Millions":
    master_table[key] = master_table[key]* (10 ** 6) #multiple every value in dataframe by 1e9 if scale is billion


#FILTER OUT YEAR COLUMNS ACCOUNTING DOESNT ADD UP
#first level: make sure 'Taxes', 'Social Contributions', 'Grants, 'Other Revenue' Sums to 'Revenue'
#second level: make sure 'Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.' sum to 'Taxes'
first_level_ls = ['Compensation of employees', 'Use of goods and services', 'Consumption of fixed capital', 'Interest', 'Subsidies', 'Grants', 'Social benefits', 'Other expense', 'Net/gross investment in nonfinancial assets']
# second_level_ls = ['Taxes on income, profits, & capital gains', 'Taxes on payroll & workforce', 'Taxes on property', 'Taxes on goods & services', 'Taxes on int trade & transactions', 'Other taxes n.e.c.']
                  
for key in master_table:
  years_accounting_error_ls = []
  years_available = master_table[key].columns
  for year in years_available:
    country_year_df = master_table[key][year]
    # revenue = 0 
    exp = 0
    for category in first_level_ls:
      if category in country_year_df.index:
        exp += country_year_df[category]
      # else: 
      #   print(category)
    # for category in second_level_ls:
    #   if category in country_year_df.index:
    #     taxes += country_year_df[category]
    # if round(taxes, 0) != round(country_year_df['Taxes'], 0) or round(revenue, 0) != round(country_year_df['Revenue'], 0):
    
    margin = 0.001
    lower_bound_exp = country_year_df['Expenditure'] - country_year_df['Expenditure']*margin
    upper_bound_exp = country_year_df['Expenditure'] + country_year_df['Expenditure']*margin

    # print(lower_bound_exp)
    # print(upper_bound_exp)
    # print(exp)
    if exp < lower_bound_exp or exp > upper_bound_exp:
    
      print(f"{round(exp, 0)}, {round(country_year_df['exp'], 0)}")

      years_accounting_error_ls.append(year)
  
  print(key)
  print(years_accounting_error_ls)
  master_table[key].drop(columns=years_accounting_error_ls, inplace=True)
 
    
    

#FILTER OUT COUNTRIES WITH LESS THAN 7 YEARS OF DATA
less_than_seven_years_data_countries = [] #create list to store countries with less than 7 years of historial data
for key in master_table: #iterate through master_table
  num_columns = master_table[key].shape[1] #count the number of columns (each column represents a year)
  if num_columns < 7: #if less than 7 columns/year, add to list to delete
    less_than_seven_years_data_countries.append(key)

print('remove')
for country in less_than_seven_years_data_countries:
  print(country)
  del master_table[country] #delete countries with less than 7 years from data table

master_table["Zambia"]

Benin
[]
Senegal
[]
Côte d'Ivoire
[]
Burkina Faso
[]
Central African Republic
[]
Ghana
[]
Somalia
[]
Tanzania
[]
Zambia
[]
Democratic Republic of the Congo
[]
Rwanda
[]
Republic of Congo
[]
Sudan
[]
The Gambia
[]
Togo
[]
Mali
[]
Malawi
[]
Sao Tomee and Principe
[]
Mozambique
[]
Sierra Leone
[]
Guinea-Bissau
[]
Ethiopia
[]
Burundi
[]
Liberia
[]
Cameroon
[]
Madagascar
[]
Uganda
[]
remove
Benin
Somalia
The Gambia
Guinea-Bissau
Liberia


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
nan,,,,,,,,,,
TRANSACTIONS AFFECTING NET WORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,1.612499e+10,2.401304e+10,2.403718e+10,2.678465e+10,3.128051e+10,3.641924e+10,4.283610e+10,4.870935e+10,5.619082e+10,6.231981e+10
Taxes,1.247751e+10,1.678054e+10,1.954949e+10,2.171162e+10,2.632927e+10,2.639580e+10,2.885596e+10,3.739301e+10,4.564338e+10,5.010771e+10
Social contributions,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Other revenue,1.485211e+09,3.138591e+09,2.331028e+09,2.858794e+09,3.301976e+09,7.155174e+09,1.032787e+10,4.103434e+09,7.770645e+09,8.768542e+09
Expense,1.426046e+10,1.652937e+10,2.170219e+10,2.701759e+10,3.383866e+10,4.260635e+10,4.842472e+10,5.152103e+10,5.533955e+10,6.596701e+10
Compensation of employees,6.394299e+09,6.950181e+09,1.066433e+10,1.137620e+10,1.536678e+10,1.639741e+10,1.864327e+10,2.002802e+10,2.096917e+10,2.233316e+10
Use of goods and services,3.440688e+09,6.507450e+09,5.394747e+09,7.762790e+09,7.342421e+09,8.956639e+09,1.425967e+10,1.244033e+10,8.807746e+09,9.273553e+09
Consumption of fixed capital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Convert Currency

In [45]:
# print('keep')
# for key in master_table:
#   print(key)

# #CONVERT LOCAL CURRENCY DENOMINATED VALUES TO USD
# error_countries = []
# for key in master_table:
#   #get annual 2010-2019 USD to domestic currency exchange rate using yfinance library
#   currency = country_currency_codes_table[key] #retrieve currency symbol from 'country_currency_codes_table'
#   symbol = f"{currency}=X"
#   try:
#     data = yf.download(symbol, start="2010-01-01", end="2020-01-01", interval="1mo") #download monthly USD-Currency dataset from 2010-2020
#     yearly_data_df = data['Close'].resample('YE').mean() #get the annual average exchange rate outputted as datafram
#     yearly_data_table = yearly_data_df.to_dict() # Convert the yearly_data to a hash table with the date as the key
#     year_to_exchange_rate_dict = {key.year: value for key, value in yearly_data_table.items()} # Convert the keys to only the year part
#     print(year_to_exchange_rate_dict)

#     #convert DataFrame values to USD
#     for year in master_table[key].columns: #loop through columns in dataframe
#       year_int = int(year)  # Convert year to integer to match the dictionary keys
#       master_table[key][year] = master_table[key][year] / year_to_exchange_rate_dict[year_int] #divide each column's values by that year's exchange rate

#   except Exception as e: #if errored out
#     print(f"Could not fetch rate for {key} ({currency}): {e}")
#     error_countries.append(key) #add country to error list
#     pass
    
# for country in error_countries:
#   del master_table[country] #delete countries where currency conversion failed
 
# for key in master_table:
#   print(key)
#   print(master_table[key])

Calculate Percentages

In [46]:
# yearly_averages_percentage_ls = []

# year = 2010
# for year_average_df in yearly_averages_ls:


#   total_taxes = year_average_df['Expenditure']
#   year_average_percentage_df = year_average_df / total_taxes * 100 #take all values as a fraction of total expenditure, then convert decimal to percent, then round to two decimal places
  
#   first_level = 0
#   print(year)
#   for category in first_level_ls:
#     first_level += year_average_percentage_df[category]
#   print(first_level)
#   year += 1
  
#   yearly_averages_percentage_ls.append(year_average_percentage_df)

# for i in range(len(yearly_averages_percentage_ls)):
#   print(f"YEAR: {2010+i}")
#   print(yearly_averages_percentage_ls[i])


for key in master_table: #iterate through master_table
  total_taxes = master_table[key].loc['Expenditure']
  master_table[key] = master_table[key] / total_taxes * 100 #take all values as a fraction of total expenditure, then convert decimal to percent, then round to two decimal places

master_table['Central African Republic']

,2010,2011,2012,2014,2015,2016,2017,2018,2019
nan,,,,,,,,,
TRANSACTIONS AFFECTING NET WORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,64.303163,64.641880,103.777902,113.745080,89.455507,104.461888,112.877803,102.120628,123.746810
Taxes,35.855305,44.657072,53.959563,24.400145,52.939240,45.812796,57.813503,52.397561,55.977026
Social contributions,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Other revenue,2.675500,5.400578,7.991041,2.874206,2.080739,3.315309,0.954819,1.080460,3.187153
Expense,69.667412,79.739346,54.039420,49.246634,90.417125,60.710637,70.840111,61.394608,66.884348
Compensation of employees,17.220627,23.707590,26.385051,34.224766,54.920374,30.603191,33.926042,30.153283,33.634773
Use of goods and services,40.341324,38.884420,12.876714,4.039469,14.761857,13.671939,18.823753,14.983236,18.805422
Consumption of fixed capital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calculate Yearly Averages

In [47]:
data_by_year_table = {
  2010: [],
  2011: [],
  2012: [],
  2013: [],
  2014: [],
  2015: [],
  2016: [],
  2017: [],
  2018: [],
  2019: [],
} #make table that stores data by year

# master_averages_ls = [] #make master averages list as place to store averages

for key in master_table:
  years_available = master_table[key].columns
  for year in data_by_year_table:
    if year in years_available:
      data_by_year_table[year].append(master_table[key][year])

yearly_averages_ls = [] #make a list of the yearly averages
for year in data_by_year_table:
  merged_year_df = pd.concat(data_by_year_table[year], axis=1, join='outer')
  #SUM OR MEAN?
  year_average_df = merged_year_df.mean(axis=1) #get the average of all the countries' averages
  yearly_averages_ls.append(year_average_df)

 
# #Get average over decade
# #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
# for key in master_table:
#   per_category_average = master_table[key].mean(axis=1) #get the average of each category (ignoring np.NaN values) into a list
#   # per_category_average.drop('To other government units', inplace=True) #drop 'To other government units' since there's two rows named that and will error
#   master_averages_ls.append(per_category_average) #add each country average to to the master averages list



# merged_averages_df = pd.concat(master_averages_ls, axis=1, join='outer')
# average__of_averages_df = merged_averages_df.mean(axis=1) #get the average of all the countries' averages
# average__of_averages_df

for i in range(len(yearly_averages_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_ls[i])

YEAR: 2010
nan
TRANSACTIONS AFFECTING NET WORTH                                       NaN
Revenue                                                          97.267387
Taxes                                                            53.256718
Social contributions                                              0.392442
Other revenue                                                    16.112018
Expense                                                          66.667672
Compensation of employees                                        22.245489
Use of goods and services                                        17.154835
Consumption of fixed capital                                           NaN
Interest                                                          4.656631
Subsidies                                                         4.357664
Grants                                                           13.691973
Social benefits                                                   1.662801
Other expe

Calculate Average Across Period

In [48]:
merged_percentages_df = pd.concat(yearly_averages_ls, axis=1, join='outer')
average_df = merged_percentages_df.mean(axis=1)
rounded_average_df = average_df.apply(lambda x: round(x, 2))

rounded_average_df = rounded_average_df.to_frame()
rounded_average_df

,0
nan,
TRANSACTIONS AFFECTING NET WORTH,NaN
Revenue,89.22
Taxes,60.84
Social contributions,0.32
Other revenue,11.75
Expense,71.65
Compensation of employees,25.51
Use of goods and services,15.94
Consumption of fixed capital,0.00


In [49]:
#check
first_level = 0
for category in first_level_ls:
  first_level += average_df[category]
print(first_level)

# second_level = 0
# for category in second_level_ls:
#   second_level += average_df[category]
# print(second_level)

99.88115226306056


In [51]:
rounded_by_year_average_df = merged_percentages_df.round(2)
rounded_by_year_average_df

,0,1,2,3,4,5,6,7,8,9
nan,,,,,,,,,,
TRANSACTIONS AFFECTING NET WORTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,97.27,90.61,88.78,93.03,88.62,84.47,84.35,86.61,87.85,90.62
Taxes,53.26,56.63,60.68,59.94,59.36,61.85,62.77,64.60,63.52,65.81
Social contributions,0.39,0.48,0.28,0.29,0.22,0.23,0.26,0.33,0.46,0.30
Other revenue,16.11,13.03,12.95,14.67,10.70,8.88,8.09,8.78,12.39,11.86
Expense,66.67,66.59,70.98,69.47,70.39,73.95,73.01,74.40,75.56,75.51
Compensation of employees,22.25,22.48,25.87,25.74,25.50,26.72,26.57,26.65,26.28,27.04
Use of goods and services,17.15,17.90,15.93,15.17,15.44,15.60,15.74,15.61,15.82,15.07
Consumption of fixed capital,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00
